# Malware Analysis & Triage Kit
This notebook performs the initial stages of immediate malware triage.

## How To
Take your malware specimen and drop it into the `dropbox` directory. The notebook will walk you through the stages of initial analysis.

At the end of this process, you will have a triage report in the `saved-specimens` diretory. This report includes findings from initial triage, including the defanged specimen in a password-proteced Zip file and static analysis artifacts.

# Imports and Setup

In [1]:
# Imports
from hashlib import *
import sys
import os
from getpass import getpass
from virus_total_apis import PublicApi as VirusTotalPublicApi
import json
from MalwareSample import *
from pprint import pprint
import os.path
from time import sleep

import pefile
import requests
import bs4

### Check Dropbox and Saved-Specimens

In [2]:
MalwareSample.check_dir("dropbox")
MalwareSample.check_dir("saved-specimens")
empty = MalwareSample.is_dir_empty("dropbox")
if empty:
    print(r"  \\--> " + recc + "Put some samples in the dropbox!")

[*] Directory 'dropbox' exists.
[*] Directory 'saved-specimens' exists.


### Enumerate Samples in the Dropbox

In [3]:
samples=!ls dropbox/*
for s in samples:
    print(info + "Sample: " + s)

[*] Sample: dropbox/SampleNegative.txt
[*] Sample: dropbox/SamplePositive.txt
[*] Sample: dropbox/Unknown.exe


In [4]:
sample_obj = [MalwareSample(s) for s in samples]

### Create a Saved Specimen directory for the specimen(s)

In [5]:
for obj in sample_obj:
    saved_sample_name = MalwareSample.create_specimen_dirs(obj.sample_name)
    obj.saved_sample_name = saved_sample_name

### Defang Sample

In [6]:
for obj in sample_obj:
    sample_path = MalwareSample.move_and_defang(obj.sample_name, obj.saved_sample_name)
    obj.sample_path = sample_path

---

## File Hashes

### SHA256 Sum

In [7]:
for obj in sample_obj:
    hash = MalwareSample.get_sha256sum(obj.sample_path, obj.saved_sample_name)
    obj.sha256sum = hash
    print(info + obj.sample_name + ": " + obj.sha256sum)

[*] SampleNegative.txt: ffabe6c6a2911b4f47edaff4b5a2ffac92b891d5949bcb2e1de66c7b9743728e
[*] SamplePositive.txt: 275a021bbfb6489e54d471899f7db9d1663fc695ec2fe2a2c4538aabf651fd0f
[*] Unknown.exe: 92730427321a1c4ccfc0d0580834daef98121efa9bb8963da332bfd6cf1fda8a


---

## String Analysis

### StringSifter
StringSifter is a FLARE developed tool that uses an ML model to rank a binary's strings by relevance to malware analysis.

In [8]:
length = int(input(recc + "Input your desired minimum string length [default is 4, 6-8 is recommended] > "))

In [9]:
for obj in sample_obj:
    MalwareSample.pull_strings(length, obj.saved_sample_name, obj.sample_path)

[*] Written to outfile: saved-specimens/01-03-2022-125541_SampleNegative.txt/StringSifter-Out.log
[*] Written to outfile: saved-specimens/01-03-2022-125541_SamplePositive.txt/StringSifter-Out.log
[*] Written to outfile: saved-specimens/01-03-2022-125541_Unknown.exe/StringSifter-Out.log


## VT Analysis
Submit samples to Virus Total and generate a malicious confidence level.

In [10]:
VT_API_KEY = getpass("Enter VirusTotal API Key (blank if none): ")

In [11]:
if VT_API_KEY:
    vt = VirusTotalPublicApi(VT_API_KEY)
else:
    print(info + "No VT API Key. Skipping...")

[*] No VT API Key. Skipping...


Note: If there are more than 4 samples in the dropbox, hashes are submitted with a sleep of 16 seconds to remain under the public API rate limit. So hit go, grab a beverage of choice, stretch out and relax. This could be a while depending on how many samples you're submitting.

In [12]:
if VT_API_KEY:
    for obj in sample_obj:
        print(info + obj.sample_name + ":")
        print(r"  \\--> " + info + "SHA256sum: " + obj.sha256sum)
        res = vt.get_file_report(obj.sha256sum)
        conf = malicious_confidence(res)
        print(r"  \\--> " + info + "Confidence level: " + str(conf))
        crit_level =  determine_criticality(conf)
        obj.criticality = crit_level
        

        if len(sample_obj) >= 5:
            sleep(16)
        
else:
    print(info + "No VT API Key. Skipping...")

[*] No VT API Key. Skipping...


## Identify PE Sample and Parse IAT
Original Code: Squiblydoo [MalAPIReader](https://github.com/Squiblydoo/MalAPIReader)

In [13]:
# MalAPI.io currently does not have an actual API
"""
TODO:
    Identify PE sample
    Parse IAT and compare to entries on MalAPI
    Retain results in object
"""
def apiCheck(api):
    APItoCheck = api
    APICheck = requests.get("https://malapi.io/winapi/" + APItoCheck)
    APICheck.raise_for_status()
    APISoup = bs4.BeautifulSoup(APICheck.text, 'html.parser')
    
    details = APISoup.select('.detail-container .content')
    ApiInfo = details[1].getText().lstrip().rstrip()
    print(ApiInfo)
    


for obj in sample_obj:
    try:
        pe = pefile.PE("./dropbox/"+obj.sample_name, fast_load=True)
        pe.parse_data_directories()
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            print("----", entry.dll.decode("utf-8"),"----")
            for imp in entry.imports:
                print(imp.name.decode("utf-8"))
                try:
                    apiCheck(imp.name.decode("utf-8"))
                except:
                    continue
    except pefile.PEFormatError:
        print(info + f"{obj.sample_name} is not a valid PE file. Skipping...")




[*] SampleNegative.txt is not a valid PE file. Skipping...
[*] SamplePositive.txt is not a valid PE file. Skipping...
---- KERNEL32.dll ----
GetModuleFileNameW
CloseHandle
CreateProcessW
GetCurrentProcessId
GetCurrentProcessId is used to retrieve the process identifier of the calling process.
GetCurrentThreadId
GetCurrentThreadId is used to retrieve the thread identifier of the calling thread.
IsProcessorFeaturePresent
GetSystemTimeAsFileTime
Retrieves the current system date and time. The information is in Coordinated Universal Time (UTC) format. This function is commonly used by malware for anti-debugging.
InitializeSListHead
IsDebuggerPresent
IsDebuggerPresent is used to determine whether the calling process is being debugged by a user-mode debugger.
TerminateProcess
TerminateProcess is used to terminate a process.
GetCurrentProcess
GetCurrentProcess is used to retrieve a handle for the current process.
GetModuleHandleW
UnhandledExceptionFilter
SetUnhandledExceptionFilter
QueryPerfo

## Zip and Password Protect

In [14]:
for obj in sample_obj:
    zip_file = MalwareSample.zip_and_password_protect(obj.sample_path, obj.saved_sample_name)
    MalwareSample.delete_unzipped_sample(obj.sample_path, zip_file)

---

### Debug Object Vars

In [15]:
for obj in sample_obj:
    pprint(vars(obj))

{'sample_name': 'SampleNegative.txt',
 'sample_path': 'saved-specimens/01-03-2022-125541_SampleNegative.txt/Malware.SampleNegative.txt.malz',
 'saved_sample_name': '01-03-2022-125541_SampleNegative.txt',
 'sha256sum': 'ffabe6c6a2911b4f47edaff4b5a2ffac92b891d5949bcb2e1de66c7b9743728e'}
{'sample_name': 'SamplePositive.txt',
 'sample_path': 'saved-specimens/01-03-2022-125541_SamplePositive.txt/Malware.SamplePositive.txt.malz',
 'saved_sample_name': '01-03-2022-125541_SamplePositive.txt',
 'sha256sum': '275a021bbfb6489e54d471899f7db9d1663fc695ec2fe2a2c4538aabf651fd0f'}
{'sample_name': 'Unknown.exe',
 'sample_path': 'saved-specimens/01-03-2022-125541_Unknown.exe/Malware.Unknown.exe.malz',
 'saved_sample_name': '01-03-2022-125541_Unknown.exe',
 'sha256sum': '92730427321a1c4ccfc0d0580834daef98121efa9bb8963da332bfd6cf1fda8a'}
